# Báo cáo cuối kỳ Thị Giác Máy Tính


## I. Giới thiệu đề tài
Trong thời đại chuyển đổi số hiện nay, việc số hóa thông tin từ các tài liệu giấy tờ như căn cước công dân (CCCD) là một nhu cầu thiết yếu, phục vụ cho các hệ thống tự động hóa quản lý, định danh và lưu trữ dữ liệu. Tuy nhiên, các tài liệu này thường được chụp dưới nhiều điều kiện khác nhau (nghiêng, mờ, ánh sáng yếu...), dẫn đến việc xử lý tự động gặp nhiều thách thức.
1. Xác định 4 góc của căn cước công dân trong ảnh để xử lý ảnh.
2. Xác định vùng chứa chữ để trích xuất các trường dữ liệu của ảnh.
3. Nhận dạng ký tự trong từng vùng, nhằm số hóa nội dung.

Hệ thống được triển khai bằng mô hình YOLOv8 để xử lý các bước phát hiện góc và vùng chữ, kết hợp với Tesseract OCR cho bước nhận dạng ký tự. Đề tài cũng được tích hợp vào một hệ thống web demo gồm frontend Vue.js và backend FastAPI để thể hiện khả năng ứng dụng thực tiễn.

In [11]:
%matplotlib inline

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

## II. Giới thiệu về tập dữ liệu
Hình ảnh Căn Cước Công Dân được thu thập từ các nguồn như Roboflow, Kaggle, Internet và tự chụp

### 1. Tập dữ liệu để xác định 4 góc của căn
Tập dữ liệu này được tải lên roboflow để gán nhãn. Mỗi ảnh có annotations gồm:
<ul>
    <li>top-left</li>
    <li>top-right</li>
    <li>bottom-left</li>
    <li>bottom-right</li>
</ul>

Link dữ liệu đã gán nhán: <a href="https://app.roboflow.com/socialv2/cornerdetection-vsjvm/models">https://app.roboflow.com/socialv2/cornerdetection-vsjvm/models</a>

In [ ]:
### display shape, size of coner

### 2. Tập dữ liệu để xác định (Mặt trước)
Tập dữ liệu này cũng được tải lên roboflow để gán nhãn. Mỗi ảnh có annotations gồm:
<ul>
    <li>id</li>
    <li>full_name</li>
    <li>date_of_birth</li>
    <li>sex</li>
    <li>nationality</li>
    <li>place_of_origin</li>
    <li>place_of_residence</li>
    <li>date_of_expiry</li>
    <li>qr_code</li>
</ul>

### 3. Tập dữ liệu để xác định (Mặt sau)
Tập dữ liệu này cũng được tải lên roboflow để gán nhãn. Mỗi ảnh có annotations gồm:
<ul>
    <li>fingerprint</li>
    <li>issue_date</li>
    <li>issue_place</li>
    <li>personal_identification</li>
</ul>
Link dữ liệu đã gán nhán: <a href="https://app.roboflow.com/socialv2/backcccd/7">https://app.roboflow.com/socialv2/backcccd/7</a>

## III. Kỹ thuật tăng cường dữ liệu
Trong quá trình huấn luyện, để cải thiện khả năng tổng quát hóa của mô hình và giảm hiện tượng overfitting, các kỹ thuật sau đây đã được áp dụng trong bài toán.

<ul>
    <li>Điều chỉnh sắc độ (Hue) trong không gian màu HSV để tạo ra màu sắc đa dạng.</li>
    <li>Điều chỉnh độ bão hòa (Saturation), làm ảnh tươi hoặc nhạt hơn.</li>
    <li>Điều chỉnh độ sáng (Brightness) của ảnh.</li>
    <li>Xoay ảnh trong khoảng ±10 độ, giúp mô hình học được các biến dạng do xoay.</li>
    <li>Tịnh tiến ảnh theo cả trục x và y với độ lệch tối đa là 10%</li>
    <li>Phóng to hoặc thu nhỏ ảnh lên đến ±50% để mô hình không phụ thuộc vào kích thước đối tượng.</li>
    <li>Biến dạng hình học bằng phép xiên (shear) ảnh ±2 độ.</li>
    <li>Xác suất lật ảnh theo chiều dọc là 50%.</li>
    <li>Xác suất lật ảnh theo chiều ngang là 50% – thường dùng cho ảnh chứa chữ.</li>
    <li>Áp dụng kỹ thuật Mosaic Augmentation – kết hợp 4 ảnh thành 1 để tăng tính đa dạng bối cảnh.</li>
    <li>Áp dụng kỹ thuật MixUp – trộn hai ảnh cùng nhãn với tỷ lệ 20% để làm mượt biên nhãn.</li>
</ul>

## IV. Mô hình
### 1. Mô hình đề xuất
Mô hình YOLOv8 (You Only Look Once version 8) – một trong những mô hình phát hiện đối tượng (object detection) hiện đại và mạnh mẽ nhất hiện nay, được phát triển bởi Ultralytics.


In [38]:
from abc import ABC, abstractmethod
from ultralytics import YOLO
import os
import numpy as np

class BaseDetector(ABC):
    def __init__(self, model="yolov8n.pt"):
        self.model = YOLO(model)

    @abstractmethod
    def train(self, data, epochs=50, imgsz=640):
        pass

    def predict(self, img):
        return self.model(img, verbose=False)

    def evaluate(self, data):
        metrics = self.model.val(data=data)
        return {
            "mean_mAP50": np.mean(metrics.box.map50),
            "mean_mAP50-95": np.mean(metrics.box.map),
            "mean_precision": np.mean(metrics.box.mp),
            "mean_recall": np.mean(metrics.box.mr)
        }

    def save(self, out="best_model.pt"):
            self.model.save(out)

    def get_model(self):
        return self.model


### 2. Mô hình phát hiện góc của CCCD
#### 2.1 CorderDetector (sử dụng yolov8n.pt)

In [39]:
from ultralytics import YOLO
import torch
import numpy as np

class CornerDetector:
    def __init__(self, model="yolov8n.pt"):
        self.device = None
        self.model = YOLO(model)

    def train(self, data, epochs=100, imgsz=640, batch = 16):
        self.model.train(data=data, epochs=epochs, imgsz=imgsz, device=self.device, amp=True, batch=batch)

    def save(self, out="best_model.pt"):
        self.model.save(out)

    def predict(self, img, save=False, verbose=False):
        return self.model(img, save=save, verbose=verbose)

    def evaluate(self, data):
        metrics = self.model.val(data=data)
        return {
            "mean_mAP50": np.mean(metrics.box.map50),
            "mean_mAP50-95": np.mean(metrics.box.map),
            "mean_precision": np.mean(metrics.box.mp),
            "mean_recall": np.mean(metrics.box.mr)
        }

    def get_model(self):
        return self.model

    def get_classes(self):
        return self.model.names

In [40]:
corner_detector = CornerDetector()

#### 2.2 Trực quan model
- 129 lớp
- 3,157,200 tham số

In [41]:
print(corner_detector.get_model())
corner_detector.get_model().info()

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

(129, 3157200, 0, 8.8575488)

### 3. Mô hình phát hiện vùng văn bản mặt trước CCCD
#### 3.1 FrontFieldDetector (sử dụng yolov8l.pt)

In [45]:
class FrontFieldDetector(BaseDetector):
    def __init__(self):
        super().__init__(model="yolov8l.pt")
    
    def train(self, data, epochs=50, imgsz=640):
        if not os.path.exists(data):
            raise FileNotFoundError(f"Không tìm thấy file {data}. Hãy kiểm tra đường dẫn!")
        
        self.model.train(
            data=data,
            epochs=epochs,
            imgsz=imgsz,
            device=self.device,
            amp=True,
            augment=True,
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            degrees=10.0,
            translate=0.1,
            scale=0.5,
            shear=2.0,
            flipud=0.5,
            fliplr=0.5,
            mosaic=1.0,
            mixup=0.2,
        )

In [46]:
front_field_detector = FrontFieldDetector()

#### 3.2 Trực quan model
- 209 lớp
- 43,691,520 tham số

In [47]:
print(front_field_detector.get_model())
front_field_detector.get_model().info()

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_r

(209, 43691520, 0, 165.742848)

### 4. Mô hình phát hiện vùng văn bản mặt sau CCCD
#### 4.1 BackFieldDetector (sử dụng yolov8n.pt)

In [48]:
class BackFieldDetector(BaseDetector):
    def __init__(self):
        super().__init__(model="yolov8n.pt")
        
    def train(self, data, epochs=50, imgsz=640):
        if not os.path.exists(data):
            raise FileNotFoundError(f"Không tìm thấy file {data}. Hãy kiểm tra đường dẫn!")
        
        self.model.train(
            data=data,
            epochs=epochs,
            imgsz=imgsz,
            device=self.device,
            amp=True,
            augment=True,
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            degrees=10.0,
            translate=0.1,
            scale=0.5,
            shear=2.0,
            flipud=0.5,
            fliplr=0.5,
            mosaic=1.0,
            mixup=0.2,
        )

In [49]:
back_field_detector = BackFieldDetector()

#### 4.2 Trực quan model

In [51]:
print(back_field_detector.get_model())
back_field_detector.get_model().info()

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

(129, 3157200, 0, 8.8575488)

## V. Huấn luyện mô hình


### 2. Tham số huấn luyện
- Thiết bị GPU: cuda

In [53]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

Device: cuda
GPU name: NVIDIA GeForce RTX 3060 Laptop GPU


#### 2.1 Tham số huấn luyện mô hình phát hiện góc của căn cước công dân
- Số epochs: 100
- Kích thước batch: 16
- Kích thước đầu vào: (640x640)

#### 2.2 Tham số huấn luyện mô hình phát hiện vùng văn bản mặt trước CCCD
- <strong>Số epochs</strong>: 100
- <strong>Kích thước batch</strong>: 16
- <strong>Kích thước đầu vào</strong>: (640x640)

#### 2.2 Tham số huấn luyện mô hình phát hiện vùng văn bản mặt sau CCCD
- <strong>Số epochs</strong>: 100
- <strong>Kích thước batch</strong>: 16
- <strong>Kích thước đầu vào</strong>: (640x640)

### 3. Quá trình huấn luyện

## VI. Đánh giá mô hình
### 1. Đánh giá dựa trên tập Test
Các chỉ số được dùng để đánh giá:
<ul>
    <li><Strong>Precision</strong>: Tỷ lệ dự đoán đúng trên tổng số dự đoán.</li>
    <li><Strong>Recall</strong>: Tỷ lệ dự đoán đúng trên tổng số thực tế.</li>
    <li><Strong>mAP@0.5 (mean Average Precision)</strong>: Trung bình của Precision ở ngưỡng IoU 0.5.</li>
    <li><Strong>mAP@0.5:0.95</strong>: Trung bình mAP ở nhiều ngưỡng IoU (0.5 đến 0.95 cách 0.05).</li>
</ul>

#### 1.1 Đánh giá mô hình phát hiện góc

In [ ]:
#Replace file .yaml
corner_result = corner_detector.evaluate('./dataset_v2/data.yaml')

print(f'\nPrecision: {corner_result['mean_precision']}')
print(f'Recall: {corner_result['mean_recall']}')
print(f'mAP50: {corner_result['mean_mAP50']}')
print(f'mAP50-95: {corner_result['mean_mAP50-95']}')

#### 1.2 Đánh giá mô hình phát hiện vùng văn bản mặt trước CCCD

In [ ]:
#Replace file .yaml
front_field_result = front_field_detector.evaluate('./dataset_v2/data.yaml')

print(f'\nPrecision: {front_field_result['mean_precision']}')
print(f'Recall: {front_field_result['mean_recall']}')
print(f'mAP50: {front_field_result['mean_mAP50']}')
print(f'mAP50-95: {front_field_result['mean_mAP50-95']}')

#### 1.3 Đánh giá mô hình phát hiện vùng văn bản mặt sau CCCD

In [ ]:
#Replace file .yaml
back_field_result = back_field_detector.evaluate('./dataset_v2/data.yaml')

print(f'\nPrecision: {back_field_result['mean_precision']}')
print(f'Recall: {back_field_result['mean_recall']}')
print(f'mAP50: {back_field_result['mean_mAP50']}')
print(f'mAP50-95: {back_field_result['mean_mAP50-95']}')

### 2. Đánh giá bằng ma trận nhầm lẫn
<ul>
    <li>Sau khi huấn luyện, mô hình được đánh giá thêm bằng quan sát kết quả đầu ra trên nhiều ảnh.</li>
    <li>
        Kiểm tra các trường hợp
        <ul>
            <li>Dự đoán đúng (TP), sai (FP), bỏ sót (FN).</li>
            <li>Mô hình xử lý tốt ảnh mờ, xoay, bị chồng chéo hay không?</li>
        </ul>
    </li>
    <li>Có thể kết hợp với confusion matrix, thống kê lỗi theo từng loại (nếu có phân lớp).</li>
</ul>

#### 2.1 Đánh giá mô hình phát hiện góc

#### 2.2 Đánh giá mô hình phát hiện vùng văn bản mặt trước CCCD

#### 2.3 Đánh giá mô hình phát hiện vùng văn bản mặt sau CCCD

### 3. Hiển thị hình ảnh CCCD kèm bounding box
#### 3.1 Kết quả nhận diện góc

#### 3.2 Kết quả nhận diện vùng văn bản mặt trước

#### 3.3 Kết quả nhận diện vùng văn bản mặt sau

## VII. Nhận dạng kí tự
### 1. Nhận dạng kí tự Tesseract

In [58]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

### 2. Nhận dạng kí tự Vietocr

In [57]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

### 3. So sánh Tesseract và Vietocr

### 4. Kết quả trích xuất Căn cước công dân

## VIII. Ứng dụng và triển khai thực tế
### 1. Kiến trúc hệ thống
Hệ thống gồm 2 thành phần chính:
<ul>
    <li>Frontend (VueJS)</li>
    <li>Backend (FastAPI)</li>
</ul>

Đường dẫn ứng dụng: <a href="https://ocr.pgonevn.com">https://ocr.pgonevn.com</a></br>
Mã nguồn: <a href="https://github.com/ngquochuydl23/VietnameseIDCard">https://github.com/ngquochuydl23/VietnameseIDCard</a>

### 2. Hình ảnh Demo
#### 2.1 Trang chủ
![image](./screenshots/screenshot_1.png)

#### 2.2 Kết quả
21112801 - Nguyễn Quốc Huy
![image](./screenshots/screenshot_2.png)

21111531 - Lê Thị Thu Hương
![image](./screenshots/screenshot_3.png)

## IX. Kết luận và Hướng phát triển
### 1. Kết luận
- Tổng kết lại
    - Bài toán đã giải quyết được những gì?
    - Các bước xử lý (xác định góc, vùng chữ, OCR).
    - Mô hình sử dụng (YOLOv8, OCR model...).
    - Kết quả đạt được (các chỉ số đánh giá, quan sát trực quan).
- Nhấn mạnh những điểm mạnh của hệ thống.
### 2. Hạn chế
- Mô hình còn nhận diện sai về trường dữ liệu.
- Nhận dạng kí tự tiếng Việt chưa được chính xác.
- Thời gian trích xuất thông tin căn cước công dân chưa nhanh.
### 3. Hướng phát triển
- Cải thiện mô hình:
    - Dùng mô hình mạnh hơn (ví dụ TrOCR thay vì CRNN).
    - Fine-tune thêm với dữ liệu thực tế.

- Tăng chất lượng dữ liệu:

    - Bổ sung ảnh từ môi trường thực tế, đa dạng ngôn ngữ/font/kích thước.

## X. Tài liệu tham khảo

[(https://docs.ultralytics.com/vi/models/yolov8/)](https://docs.ultralytics.com/vi/models/yolov8/)

[(https://miai.vn/2020/07/04/co-ban-ve-object-detection-voi-r-cnn-fast-r-cnn-faster-r-cnn-va-yolo/)](https://miai.vn/2020/07/04/co-ban-ve-object-detection-voi-r-cnn-fast-r-cnn-faster-r-cnn-va-yolo/)

[(https://cmcts.com.vn/vi/nhan-dien-the-cccd-gan-chip.html)](https://cmcts.com.vn/vi/nhan-dien-the-cccd-gan-chip.html)

[(https://github.com/pbcquoc/vietocr)](https://github.com/pbcquoc/vietocr)
